In [16]:
from collections import defaultdict
from dataclasses import dataclass
import os
import sys
import yaml
import time
import numpy as np
import scipy.io
import scipy.linalg
import scipy.sparse as sp
import matplotlib.pyplot as plt
from src.common import NDArrayFloat
from src.linalg import get_numpy_eigenvalues
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from scipy.interpolate import RectBivariateSpline

In [17]:
def generate_matrix(type: str, sparse: bool, size: int, min_val: float, max_val: float, grouping_factor: float = 0, density: float = 1) -> np.array:
  """Generates various types of REAL matrices with controlled value range and diagonal grouping.

  Args:
    type: The type of matrix to generate.
          Options: 'nonsymmetric', 'symmetric', 'normal', 'orthogonal'.
    sparse: Whether to generate a sparse matrix.
    size: The size of the matrix (size x size).
    min_val: Minimum value of the matrix elements.
    max_val: Maximum value of the matrix elements.
    grouping_factor: Controls the strength of diagonal grouping. 
                     0: No grouping.
                     Positive values: Increase grouping strength.

  Returns:
    A NumPy array representing the generated matrix.
  """
  
  seed = 42
  np.random.seed(seed)
  
  if type == 'nonsymmetric':
    if sparse:
      A = sp.rand(size, size, density=density, format='csr').toarray()
    else:
      A = np.random.rand(size, size)
    if grouping_factor > 0:
      for i in range(size):
        A[i, :] = A[i, :] * (1 + grouping_factor * i / (size-1)) # Scale based on distance and factor
    return A * (max_val - min_val) + min_val

  elif type == 'symmetric':
    if sparse:
      A = sp.rand(size, size, density=density, format='csr')
      A = (A + A.transpose()).toarray() / 2
    else:
      A = np.random.rand(size, size)
      A = (A + A.T) / 2
    if grouping_factor > 0:
      for i in range(size):
        for j in range(size):
          A[i, j] = A[i, j] * np.exp(-grouping_factor * abs(i - j) / (size - 1)) # Exponential decay from diagonal
    return A * (max_val - min_val) + min_val

  elif type == 'normal':
    if sparse:
      raise ValueError("Sparse normal matrices are not well-defined.")
    else:
      # Generate a symmetric matrix (guaranteed to be normal)
      A = np.random.rand(size, size)
      A = (A + A.T) / 2
      if grouping_factor > 0:
        for i in range(size):
          for j in range(size):
            A[i, j] = A[i, j] * np.exp(-grouping_factor * abs(i - j) / (size - 1)) # Exponential decay from diagonal
      return A * (max_val - min_val) + min_val

  elif type == 'orthogonal':
    if sparse:
      raise ValueError("Sparse orthogonal matrices are not well-defined.")
    else:
      Q, _ = np.linalg.qr(np.random.rand(size, size))
      if grouping_factor > 0:
        for i in range(size):
          Q[i, :] = Q[i, :] * (1 + grouping_factor * i / (size-1)) # Scale based on distance and factor
      return Q * (max_val - min_val) + min_val

  else:
    raise ValueError(f"Invalid matrix type: {type}")

In [18]:
nonsymmetric_matrix_sparse = generate_matrix('nonsymmetric', True, 500, 100, 1000,density=0.05)
nonsymmetric_matrix_bottom_group = generate_matrix('nonsymmetric', False, 500, 100, 1000,grouping_factor=1000)
symmetric_matrix = generate_matrix('symmetric', False, 500, 0, 10)
symmetric_matrix_diag = generate_matrix('symmetric', False, 500, 0, 10,grouping_factor=5)
orthogonal_matrix = generate_matrix('orthogonal', False, 500, 0, 1)
symmetric_matrix_diag_sparse = generate_matrix('symmetric', True, 1000, 0, 1000,grouping_factor=3,density=0.3)
symmetric_matrix_small = generate_matrix('symmetric', False, 25, 0, 1)

matrix_dict = {
    1 : [nonsymmetric_matrix_sparse, "nonsymmetric_matrix_sparse"],
    2 : [nonsymmetric_matrix_bottom_group, "nonsymmetric_matrix_bottom_group"],
    3 : [symmetric_matrix,"symmetric_matrix"],
    4 : [symmetric_matrix_diag, "symmetric_matrix_diag"],
    5 : [orthogonal_matrix, "orthogonal_matrix"],
    6 : [symmetric_matrix_diag_sparse, "symmetric_matrix_diag_sparse"],
    7 : [symmetric_matrix_small, "symmetric_matrix_small"],
}

In [19]:
def testing(my_function, A: np.array, name: str,show_plots: bool=False, function_name: str="My_func"):
  """Compares your eigenvalue algorithm with NumPy's eigvals.

  Args:
    my_function: function that takes a NumPy array (matrix) and returns a NumPy array of eigenvalues.
    A: The input matrix for eigenvalue calculation.
  """


  start_time = time.time()
  A_k = A.copy()
  for k in range(100):
      Q,R = my_function(A_k)
      A_k = R @ Q
  my_eigenvalues = np.diag(A_k)
  my_time = time.time() - start_time

  start_time = time.time()
  numpy_eigenvalues = np.linalg.eigvals(A)
  numpy_time = time.time() - start_time

  my_eigenvalues.sort()
  numpy_eigenvalues.sort()

  # Accuracy
  
  errors = np.abs(numpy_eigenvalues - my_eigenvalues) / np.abs(numpy_eigenvalues)
  max_error = np.max(errors)
  avg_error = np.mean(errors)
  median_error = np.median(errors)
  print("----------")
  print(f'{name}:')
  print("  Time:", my_time, "seconds")
  print("NumPy eigvals:")
  print("  Time:", numpy_time, "seconds")
  print("\nAccuracy:")
  print("  Maximum Error:", max_error)
  print("  Average Error:", avg_error)
  print("  Median Error:", median_error)
  print("----------\n")
  
  if(show_plots):
    # Plot the differences
    plt.figure(figsize=(8, 6))
    plt.plot(my_eigenvalues, label=function_name)
    plt.plot(numpy_eigenvalues, label="NumPy eigvals")
    plt.xlabel("Eigenvalue Index")
    plt.ylabel("Eigenvalue")
    plt.title("Eigenvalue Comparison")
    plt.legend()
    plt.show()

    # Plot the errors
    plt.figure(figsize=(8, 6))
    plt.plot(errors)
    plt.xlabel("Eigenvalue Index")
    plt.ylabel("Absolute Error")
    plt.title("Eigenvalue Error $\dfrac{\mid\lambda_{n} - \lambda_{f}\mid}{\mid\lambda_{n}\mid}$")
    plt.show()
    
  return list(max_error,avg_error,median_error)

<>:54: SyntaxWarning: invalid escape sequence '\d'
<>:54: SyntaxWarning: invalid escape sequence '\d'
/var/folders/ls/btrdy60n12n_lk3plpt7wqlh0000gn/T/ipykernel_82865/614904529.py:54: SyntaxWarning: invalid escape sequence '\d'
  plt.title("Eigenvalue Error $\dfrac{\mid\lambda_{n} - \lambda_{f}\mid}{\mid\lambda_{n}\mid}$")


In [20]:
def test_all(matrix_dict: dict, my_function,show_plots: bool=False,skip_matrices_id: list=[], function_name: str="My_function"):
    errors_all_matrices = np.zeros((3,len(matrix_dict) - len(skip_matrices_id)))
    print(errors_by_matrix)
    for matrix_id,items in matrix_dict.items():
        if(matrix_id in skip_matrices_id): continue
        name = items[1]
        matrix = items[0]
        errors_by_matrix = testing(my_function=my_function,A=matrix,name=name,show_plots=show_plots, function_name=function_name)
        errors_all_matrices[matrix_id - 1] = errors_by_matrix
        if(show_plots):
            plot_2d_temperature(matrix=matrix,title=name)
    return errors_by_matrix

In [21]:
def Gram_Schmidt_optimised_QR(A: np.array):
    """
    Optimized Gram-Schmidt process with normalization.
    The coefficients q will be calculated at each step of the algorithm.
    Each time we will subtract the component of the vector from all the vectors q at once.
    """
    m, n = A.shape  # Get both dimensions of A
    Q = A.copy()
    R = np.zeros((n, n))  # Initialize R with zeros

    for k in range(n): # Orthogonalize the k-th column
        R[k, k] = np.linalg.norm(Q[:, k]) # Normalize the k-th column
        Q[:, k] /= R[k, k] 
        for j in range(k + 1, n): # Subtract the projection from subsequent columns 
            R[k, j] = np.dot(Q[:, k], Q[:, j])
            Q[:, j] -= R[k, j] * Q[:, k] 
    return Q, R

In [23]:
errors_matrix = test_all(matrix_dict=matrix_dict, my_function=QR_eigenvalues_algorithm,function_name="Gramm optimised")

UnboundLocalError: cannot access local variable 'errors_by_matrix' where it is not associated with a value